In [3]:
import pandas as pd
import pyodbc
import pgeocode
import useful_functions as use

In [3]:
username = 'vigrose'
password_edw = 'Ravenclaw~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [4]:
ppd = use.get_ppd()
ppd.columns

C:/Users/vigrose/Data/PPD\ppd_data_20210731.csv


C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [8]:
ppd.groupby('SUFFIX_CENSUS').count()

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
SUFFIX_CENSUS,,,,,,,,,,,,,,,,,,,,,
0.0,604382,604382,604382,595903,604382,604371,604374,483943,18099,132405,...,462036,462041,456555,458415,132464,111165,132464,14823,11850,14824
1.0,157377,157377,157377,155590,157377,157374,157374,126839,4979,27269,...,119157,119157,117930,118326,36672,30800,36673,4270,3443,4270
2.0,146939,146939,146939,145437,146939,146935,146936,118320,4398,22537,...,110290,110290,109034,109356,34186,28669,34185,3916,3101,3916
3.0,59973,59973,59973,59324,59973,59973,59973,48411,1865,6551,...,44177,44175,43667,43811,14499,12097,14499,1680,1309,1680
4.0,47851,47851,47851,47560,47851,47848,47850,38644,1424,4350,...,35159,35160,34768,34911,11717,9858,11717,1381,1108,1381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95.0,88,88,88,88,88,88,88,65,1,4,...,74,74,73,73,12,10,12,2,2,2
96.0,74,74,74,73,74,74,74,58,0,1,...,44,44,43,44,11,9,11,1,1,1
97.0,69,69,69,68,69,69,69,61,3,1,...,48,48,48,48,14,13,14,2,2,2


In [ ]:
#Get ppd
# ppd = use.get_ppd()
# ppd['ME'] = fix_me(ppd.ME)
ppd[ppd.ME=='01606631029'][['TOP_CD','MAILING_LINE_2','CITY','STATE','']]

In [ ]:
Joel G. Sacks, MD ME # 01606631029 

In [ ]:
#Get party ids
ME_QUERY = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    AND
    P.KEY_VAL = '01606631029'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [ ]:
party_ids

In [10]:
#Get polos
sql_query_polo = \
    '''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL;
    '''
POLOS = pd.read_sql(con=AMAEDW, sql=sql_query_polo)
POLOS

,PARTY_ID,POST_CD_ID
0,1816025,9128232
1,2171199,9128232
2,2390541,9128232
3,2450538,9128232
4,2450602,9128232
...,...,...
1160641,3559749,32419498
1160642,6077003,32419499
1160643,3499718,32421325
1160644,2050705,32421343


In [ ]:
#Get polos
sql_query_polo_2 = \
    '''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID,
    A.CAT_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE,
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS COMM_ID
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL;
    '''
POLOS_2 = pd.read_sql(con=AMAEDW, sql=sql_query_polo_2)

In [ ]:
#Get polos
other_polo = \
    '''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE,
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS POST_KEY,
    CC.DESC,
    CC.CATEGORY_CODE
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.CAT_CD CC
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL
    AND
    CC.CAT_CD_ID = A.CAT_CD_ID;
    '''
OTHER_POLOS = pd.read_sql(con=AMAEDW, sql=other_polo)

In [ ]:
#Get ppmas
sql_query_ppma = \
    '''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE_CD,
    P.POST_CD AS ZIP,
    A.FROM_DT,
    A.THRU_DT
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PP'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.PARTY_ID = '1939823'

        '''
PPMAS = pd.read_sql(con=AMAEDW, sql=sql_query_ppma)

In [ ]:
PPMAS

In [ ]:
#Merge ppma, polo
addresses = pd.merge(POLOS, PPMAS, on='PARTY_ID', suffixes = ['_POLO','_PPMA'])

In [ ]:
addresses.head()

In [ ]:
#Add party ids and identify missing POLOs
PPD = pd.merge(ppd, party_ids, on = 'ME')
PPD = PPD.fillna('None')
no_polo = PPD[(PPD.POLO_MAILING_LINE_1=='None')&(PPD.POLO_MAILING_LINE_2=='None')]
print(len(no_polo))

In [ ]:
#Find filtered
filtered_polos = pd.merge(no_polo, addresses, on='PARTY_ID')
print(len(filtered_polos))

In [ ]:
filtered_dpc = filtered_polos[filtered_polos.TOP_CD == 20]

In [ ]:
newers = []
for row in filtered_dpc.itertuples():
    newer = False
    if row.FROM_DT_POLO > row.FROM_DT_PPMA:
        newer = True
    newers.append(newer)

In [ ]:
filtered_dpc = filtered_dpc.rename(columns ={'STATE_CD':'STATE_PPMA','STATE_x':'STATE','STATE_y':'STATE_POLO'})
filtered_dpc['POLO_NEWER'] = newers
newer = filtered_dpc[filtered_dpc.POLO_NEWER == True]

In [ ]:
len(newer)

### How many POLO addresses are being filtered out at present?
#### 85,287
### How many of those belong to DPC physicians?
#### 59,321
### For the DPC physicians, how many of the POLO addresses are newer than their PPMA addresses?
#### 36,127

In [ ]:
newer[['FROM_DT_PPMA','FROM_DT_POLO','POLO_STATE','STATE_POLO','STATE', 'STATE_PPMA']]

In [ ]:
filtered_dpc[filtered_dpc.POLO_NEWER == True]

In [ ]:
len(ppd)

In [ ]:
no_polo[no_polo.TOP_CD == 20]

In [ ]:
ppd[ppd.TOP_CD == 20]

In [ ]:
newer

In [6]:
# import pandas as pd
#Sourcing
sql_query = \
        f'''
        SELECT DISTINCT
        A.PARTY_ID,
        A.POST_CD_ID,
        CC.DESC,
        CC.CATEGORY_CODE
        FROM
        AMAEDW.POST_CD P, AMAEDW.PARTY_ADDR A, AMAEDW.CAT_CD CC, AMAEDW.CONT_PURPOSE_TYPE C
        WHERE
        CC.CAT_CD_ID = A.CAT_CD_ID
        AND
        A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
        AND
        C.PURPOSE_CAT_CD='A'
        AND
        C.PURPOSE_USG_CD='PO'
        AND
        A.THRU_DT IS NULL
        '''
sourcing = pd.read_sql(con=AMAEDW, sql=sql_query)

In [8]:
sourcing.groupby('CATEGORY_CODE').count()

,PARTY_ID,POST_CD_ID,DESC
CATEGORY_CODE,,,
ACS,9474,9474,9474
ACSIMB-MMX,1774,1774,1774
ACXIOM-DSF,6386,6386,6386
ACXIOMLODE,2,2,2
ACXIOMNCOA,2943,2943,2943
ADDR-VER,2,2,2
ADMIT-HOS,1601,1601,1601
AMC,2298,2298,2298
CGMT-EXC,1,1,1


In [ ]:
pd.merge(sourcing, PPMAS, on='POST_CD_ID')

In [ ]:
pd.merge(sourcing, newer, on='PARTY_ID')

In [ ]:
len(newer)

In [ ]:
newer.drop_duplicates('ME')

In [ ]:
len(parties)

In [ ]:
OTHER_POLOS

In [ ]:
POLOS_2

In [ ]:
len(ppd)-len(no_polo)

In [ ]:
480542/1157774

In [ ]:
entity_ids = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = '9'
    """
ENTITY_ID = pd.read_sql(con=AMAEDW, sql=entity_ids)

In [ ]:
ENTITY_ID

In [ ]:
newer = pd.merge(ENTITY_ID, newer, on='PARTY_ID')
entity_list = tuple(newer.ENTITY_ID)

In [ ]:
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [ ]:
query_3 = \
    f'''
    SELECT DISTINCT
    U.ENTITY_ID,
    U.COMM_ID,
    U.SRC_CAT_CODE
    FROM
    informix.ENTITY_COMM_USG_AT U
    WHERE
    U.COMM_USAGE = 'PO'
    AND
    U.ENTITY_ID in {entity_list}
    AND
    U.END_DT IS NULL
    ;
    '''
aims_check = pd.read_sql(con=informix, sql=query_3)
aims_check.head()

In [ ]:
newer = pd.merge(aims_check, newer, on='ENTITY_ID')

In [ ]:
newer.dtypes

In [ ]:
aims_check['ENTITY_ID'] = aims_check.entity_id.astype(str)

In [ ]:
pd.merge(newer, sourcing, on='PARTY_ID', how='left').groupby(['src_cat_code','DESC']).count()

In [ ]:
pd.merge(newer, sourcing, on='PARTY_ID', how='left').groupby('DESC').count()

In [ ]:
diffs = []
for row in newer.itertuples():
    diff = row.FROM_DT_POLO - row.FROM_DT_PPMA
    diffs.append(diff)

In [ ]:
[x.year for x in newer.FROM_DT_POLO]

In [ ]:
newer = newer.fillna('None')

In [ ]:
newer = pd.merge(newer, sourcing, on='PARTY_ID', how='left')

In [ ]:
SOURCES  = []
for row in newer.itertuples():
    if row.DESC == 'None':
        source = row.src_cat_code
    else:
        source = row.DESC
    SOURCES.append(source)

In [ ]:
newer['SOURCE']=SOURCES

In [ ]:
newer.groupby('src_cat_code').count()['entity_id'].to_csv('../../Data/PPD/POLO_Sources.csv')

In [ ]:
newer.groupby('src_cat_code').count()[['entity_id']].plot.pie(y='entity_id', figsize=(10, 10))

In [ ]:
def get_zip_distance(zip1, zip2, dist):
    '''Get distances between zipcodes'''
    zip1 = clean_zipcode(zip1)
    zip2 = clean_zipcode(zip2)
    distance = dist.query_postal_code(zip1, zip2)*0.621371
    return distance

def clean_zipcode(zipcode):
    '''Format zipcode'''
    zipcode = str(zipcode)
    zipcode = zipcode.replace(" ", "")
    zipcode = zipcode.replace(".0", "")
    if len(zipcode) == 3:
        zipcode = "00"+zipcode
    if len(zipcode) == 4:
        zipcode = "0"+zipcode
    else:
        zipcode = zipcode[0:5]
    return zipcode

def get_zip_distances(df):
    '''Get all zip distances'''
    dists = []
    print("Removing significantly close addresses...")
    dist = pgeocode.GeoDistance('US')
    for row in df.itertuples():
        dists.append(get_zip_distance(row.ZIP_POLO, row.ZIP_PPMA, dist))
    df['POLO_DISTANCE'] = dists
    return df

In [ ]:
filtered_polos[['ZIP_POLO','ZIP_PPMA']]

In [ ]:
filtered_polos = get_zip_distances(filtered_polos)

In [ ]:
filtered_polos.sort_values('POLO_DISTANCE')

In [ ]:
filtered_polos.hist(bins=3, column='POLO_DISTANCE')

In [ ]:
filtered_polos.dropna(subset=['POLO_DISTANCE']).hist(bins=20, column='POLO_DISTANCE', color='indigo')

In [ ]:
filtered_polos.dropna(subset=['POLO_DISTANCE']).POLO_DISTANCE.median()

In [ ]:
PPD[PPD.TOP_CD == 20]

In [ ]:
(819579-98272)/819579

In [ ]:
(819579-98272+20994)/819579

In [ ]:
0.9057101267846053*0.7706

In [ ]:
(819579-98272)*0.7677

In [ ]:
553747.3839+20994

In [ ]:
(574741.3839)/819579

In [ ]:
591966/819579

In [ ]:
(819579-98272+36127)/819579

In [ ]:
0.9241744847049522*0.7706

In [ ]:
newer.columns

In [ ]:
(sum([x.days for x in diffs])/len(diffs))/365.25

In [ ]:
import datetime

today = datetime.datetime.today()

In [ ]:
today

In [ ]:
polo_ages = []
ppma_ages = []
for row in newer.itertuples():
    polo_age = today - row.FROM_DT_POLO
    ppma_age = today - row.FROM_DT_PPMA
    polo_ages.append(polo_age)
    ppma_ages.append(ppma_age)

In [ ]:
(sum([x.days for x in polo_ages])/len(polo_ages))/365.25

In [ ]:
(sum([x.days for x in ppma_ages])/len(ppma_ages))/365.25

In [ ]:
newer['PPMA_AGE'] = [(x.days)/365.25 for x in ppma_ages]
newer['POLO_AGE'] = [(x.days)/365.25 for x in polo_ages]
newer['AGE_DIFFERENCE'] = [(x.days)/365.25 for x in diffs]

In [ ]:
newer.hist(bins=10, column='PPMA_AGE', color='indigo')

In [ ]:
humach = pd.read_csv('../../Data/Humach/humach_updated_since_2018.csv')

In [ ]:
humach['ME']=fix_me(humach.PHYSICIAN_ME_NUMBER)

In [ ]:
MERGE = pd.merge(newer, humach, right_on = ['ME','OFFICE_ADDRESS_STATE'], left_on=['ME','STATE_POLO'])

In [ ]:
MERGE[['ZIP_POLO','OFFICE_ADDRESS_ZIP']]

In [ ]:
newer['OFFICE_ADDRESS_LINE_2'] = [x.upper().replace('.','') for x in newer.ADDR_1_POLO]

In [ ]:
true_merge.drop_duplicates('ME').groupby('src_cat_code').count()

In [ ]:
2+5+150+1+17+1+227+1+2+34+1+9+2

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
    elif fuzz.ratio(thing_1, thing_2) > 70:
        return 1

In [ ]:
def check_match(thing_1, thing_2):
    '''match check'''
    if thing_1 == thing_2:
        return 1
    elif thing_1 in thing_2 or thing_2 in thing_1:
        return 1
    elif fuzz.ratio(thing_1, thing_2) > 60:
        return 1
    else:
        return 0

In [ ]:
MERGE = MERGE.fillna('None')

In [ ]:
corrects =[]
for row in MERGE.itertuples():
    correct = True
    one = row.OFFICE_ADDRESS_LINE_2_x
    two = row.OFFICE_ADDRESS_LINE_2_y
    if check_match(one,two) == 0:
        print(f'{one} is not {two}')
        print(fuzz.ratio(one, two))
        print('')
        correct = False
    corrects.append(correct)

In [ ]:
MERGE['VALIDATED']=corrects

In [ ]:
true_merge = MERGE[MERGE.VALIDATED==True]

In [ ]:
ppd[ppd.FIRST_NAME=="DE'ANDRE"].POLO_MAILING_LINE_2

In [ ]:
ppd.groupby('TOP_CD').count()

In [ ]:
for col in true_merge.columns:
    print(col)

In [ ]:
true_merge.sort_values('WSLIVE_FILE_DT')

In [ ]:
true_merge['FILE_DATE'] = pd.to_datetime(true_merge.WSLIVE_FILE_DT)

In [ ]:
goop = true_merge.sort_values('FILE_DATE').drop_duplicates('ME', keep='last')

In [ ]:
goop['FILE_YEAR'] = [x.year for x in goop.FILE_DATE]

In [ ]:
goop.groupby('FILE_YEAR').count()[['entity_id']].plot(kind='bar', legend=False, color='indigo', xlabel='Year', ylabel='Count', title = 'Most Recent Verified Date')

In [ ]:
goop

In [ ]:
MERGE

In [ ]:
filtered_dpc[filtered_dpc.POLO_NEWER == False].to_csv('../../Data/POLO_Filter/Filtered_POLOs_Older.csv', index=False)